# Abstractive summaries - Train DistilBART on TWEETSUMM dataset, continued

In [1]:
!ls /kaggle/input/

bart-res-p1-2409  bertdata2207	gptdata


In [2]:
!cp -r /kaggle/input/bart-res-p1-2409/* /kaggle/working

In [3]:
from huggingface_hub import login
import pandas as pd
import numpy as np
import os, time, datetime, shutil

from datasets import Dataset, DatasetDict

from transformers import DataCollatorForSeq2Seq, AutoTokenizer, set_seed
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import TrainerCallback, TrainingArguments, TrainerState, TrainerControl

import wandb

In [4]:
!pip freeze > requirements_bart.txt

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [5]:
def get_current_time(underscore=False):
    return datetime.datetime.now().strftime("%d%m-%H%M" if not underscore else "%d%m_%H%M")

In [6]:
run_name = f"bart-abs-{get_current_time()}"
models_dir = os.path.join(os.getcwd(), 'models')
os.makedirs(models_dir, exist_ok=True)
results_dir = os.path.join(os.getcwd(), 'results', 'bart')
os.makedirs(results_dir, exist_ok=True)
ds_dir = os.path.join(os.getcwd(), 'data')
print(run_name)

bart-abs-2409-1947


In [7]:
try:
    HF_TOKEN =  os.environ['HF_TOKEN']
except:
    HF_TOKEN = ""

if 'google.colab' in str(get_ipython()):
    print("Running on Colab")
    from google.colab import drive, userdata
    drive.mount('/content/drive')
    HF_TOKEN = userdata.get('HF_TOKEN')
elif os.environ.get('KAGGLE_KERNEL_RUN_TYPE') != None:
    ds_dir = '/kaggle/input/bertdata2207/'
    from kaggle_secrets import UserSecretsClient
    print("Running on Kaggle")
    user_secrets = UserSecretsClient()
    HF_TOKEN = user_secrets.get_secret("HF_TOKEN")
    WANDB_API_KEY = user_secrets.get_secret("WANDB_API_KEY")
    os.environ['WANDB_API_KEY'] = WANDB_API_KEY
    os.makedirs(os.path.join(os.getcwd(), "results"), exist_ok=True)
    os.makedirs(os.path.join(os.getcwd(), 'results', 'bart'), exist_ok=True)


Running on Kaggle


In [8]:
set_seed(17)

In [9]:
os.environ["WANDB_PROJECT"] = f"aiml-thesis-train-{run_name}"
os.environ["WANDB_WATCH"] = "false"
wandb.init(settings=wandb.Settings(start_method="thread"), id=run_name)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: dawidk5 (dawidk5ul). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.18.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240924_194754-bart-abs-2409-1947
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run bart-abs-2409-1947
wandb: ⭐️ View project at https://wandb.ai/dawidk5ul/aiml-thesis-train-bart-abs-2409-1947
wandb: 🚀 View run at https://wandb.ai/dawidk5ul/aiml-thesis-train-bart-abs-2409-1947/runs/bart-abs-2409-1947


In [10]:
login(token=HF_TOKEN)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## Load data

In [11]:
print(ds_dir)

/kaggle/input/bertdata2207/


In [12]:
checkpoint_bart = "sshleifer/distilbart-xsum-12-6"

In [13]:
def csv_to_pandas(file_name, ds_dir, drop_conv_id=True):
    df = pd.read_csv(os.path.join(ds_dir, file_name), names=['conv_id', 'dialogue', 'summary'], encoding='utf-8', dtype={'conv_id': 'string', 'dialogue': 'string', 'summary': 'string'})
    df = df.convert_dtypes()
    if drop_conv_id:
        df.drop(columns=['conv_id'], inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

In [14]:
train_df_temp = csv_to_pandas("dials_abs_2607_1312_train_spc.csv", ds_dir)
val_df_temp = csv_to_pandas("dials_abs_2607_1312_valid_spc.csv", ds_dir)
test_df = csv_to_pandas("dials_abs_2607_1312_test_spc.csv", ds_dir, drop_conv_id=False)

print(train_df_temp.dtypes)
print(train_df_temp.head(), len(train_df_temp))

dialogue    string[python]
summary     string[python]
dtype: object
                                            dialogue  \
0  Customer: So neither my iPhone nor my Apple Wa...   
1  Customer: @115850 hi team! i m planning to get...   
2  Customer: @AskAmex Where do I write to address...   
3  Customer: @AmazonHelp @115821 Wow, expected 4 ...   
4  Customer: @GWRHelp I'd rather you spent some t...   

                                             summary  
0  Customer enquired about his Iphone and Apple w...  
1  Customer is eager to know about the replacemen...  
2  Signed up for an AmexCard with Delta but it di...  
3  The customer have a problem. The agent is very...  
4  Customer cannot purchase a train ticket on the...   867


In [15]:
tweetsumm_abs = DatasetDict(
    {
        'train': Dataset.from_pandas(train_df_temp),
        'validation': Dataset.from_pandas(val_df_temp),
        'test': Dataset.from_pandas(test_df)
    }
)

In [16]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint_bart)
print(tokenizer)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

BartTokenizerFast(name_or_path='sshleifer/distilbart-xsum-12-6', vocab_size=50265, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True, special=True),
}


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [17]:
# Source: https://huggingface.co/docs/transformers/en/tasks/summarization

def preprocess_function(examples):
    prefix = "summarize: "
    inputs = [str(prefix) + str(dial) for dial in examples["dialogue"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True) # same params as tweetsumm paper
    labels = tokenizer(text_target=examples["summary"], max_length=80, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [18]:
tokenized_tweetsumm_abs = tweetsumm_abs.map(preprocess_function, batched=True, remove_columns=['dialogue','summary'])
print(tokenized_tweetsumm_abs["train"][1])

Map:   0%|          | 0/867 [00:00<?, ? examples/s]

Map:   0%|          | 0/110 [00:00<?, ? examples/s]

Map:   0%|          | 0/109 [00:00<?, ? examples/s]

{'input_ids': [0, 18581, 3916, 2072, 35, 19458, 35, 787, 1225, 4432, 1096, 20280, 165, 328, 939, 475, 1884, 7, 120, 1257, 1754, 510, 20529, 27785, 24, 924, 15, 5, 998, 24, 34, 158, 360, 5010, 21784, 6, 64, 1717, 3922, 162, 99, 16, 24, 17487, 50118, 45443, 35, 787, 2481, 3897, 2036, 166, 348, 10, 158, 7033, 5010, 714, 114, 5, 6880, 47, 829, 16, 5009, 50, 31559, 4, 37249, 10237, 50118, 44799, 35, 787, 25146, 28780, 5148, 27785, 125, 99, 114, 939, 399, 17, 27, 90, 101, 5, 1152, 8, 236, 7, 671, 24, 50118, 45443, 35, 787, 2481, 3897, 2036, 166, 1979, 75, 28, 441, 7, 3264, 5, 23312, 2886, 4, 286, 55, 335, 15, 1830, 2886, 714, 4, 17161, 352, 3753, 15, 5, 3104, 1373, 259, 35, 1205, 640, 90, 4, 876, 73, 571, 40969, 9380, 530, 4154, 510, 975, 4, 3166, 19954, 877, 110, 2969, 4, 50118, 44799, 35, 787, 25146, 28780, 5148, 2446, 27785, 125, 209, 32, 5567, 15797, 98, 473, 24, 1266, 276, 714, 3253, 13, 209, 25, 157, 50118, 45443, 35, 787, 2481, 3897, 2036, 3216, 6, 30845, 73, 5567, 15797, 32, 45, 4973

## Setup Training Evaluation

In [19]:
!pip install -U nltk

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 23.9 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.9.1 which is incompatible.


In [20]:
!pip install evaluate pyrouge rouge_score bert_score meteor

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.2/100.2 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 62.2 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - \ | / - done
  Getting requirements to build wheel ... - \ | / - done
  Preparing metadata (pyproject.toml) ... - \ done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 80.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 746.6/746.6 kB 41.1 M

In [21]:
import evaluate, nltk, csv
rouge = evaluate.load("rouge")
meteor = evaluate.load("meteor")
bertscore = evaluate.load("bertscore")

nltk.download('punkt_tab')

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [22]:
# Main source, official HUggingFace documentation: https://huggingface.co/docs/transformers/tasks/summarization
def compute_metrics_abs(eval_pred):
    predictions, labels = eval_pred
    # Extra line added to address an overflow: https://github.com/huggingface/transformers/issues/22634
    predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]

    rouge_scores = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    bert_scores = bertscore.compute(predictions=decoded_preds, references=decoded_labels, lang="en")
    bert_scores.pop('hashcode')
    result = {
      **{f"rouge/{k}": round(v, 4) for k,v in rouge_scores.items()},
      **{f"bertscore/bertscore-{k}": round(np.mean(v), 4) for k,v in bert_scores.items()},
      'meteor': round(meteor.compute(predictions=decoded_preds, references=decoded_labels)['meteor'], 4),
    }
   
    result["gen_len"] = np.mean(prediction_lens)
    return result


## Train and Evaluate

In [23]:
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_bart)

pytorch_model.bin:   0%|          | 0.00/611M [00:00<?, ?B/s]

In [24]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [25]:
EXPERIMENT_PARAMS = []
BASE_PARAMS = {'lr':3e-5, 'batch_size':4, 'epochs': 6}
EXPERIMENT_PARAMS.append(BASE_PARAMS)

In [26]:
LEARN_RATES = (3e-5, 3e-4, 3e-6)
BATCH_SIZES = (4, 2, 8)
EPOCHS = (6,10)

for lr in LEARN_RATES:
    for batch_size in BATCH_SIZES:
        for epoch in EPOCHS:
            if lr == BASE_PARAMS['lr'] and batch_size == BASE_PARAMS['batch_size'] and epoch == BASE_PARAMS['epochs']:
                continue
            experiment = {'lr':lr, 'batch_size':batch_size, 'epochs': epoch}
            EXPERIMENT_PARAMS.append(experiment)

In [27]:
os.listdir(results_dir)

['log_bart_abs_2409_0144_lr_3e_05_bs_4_maxep_10.csv',
 'test_metrics_bart_abs_2409_0144_lr_3e_05_bs_4_maxep_6_s1085_bart.csv',
 'test_metrics_bart_abs_2409_0144_lr_3e_05_bs_4_maxep_10_s1302_bart.csv',
 'test_preds_bart_abs_2409_0144_lr_3e_05_bs_4_maxep_6_s1085_bart.csv',
 'log_bart_abs_2409_0144_lr_3e_05_bs_2_maxep_6.csv',
 'test_preds_bart_abs_2409_0144_lr_3e_05_bs_4_maxep_10_s1302_bart.csv',
 'test_preds_bart_abs_2409_0144_lr_3e_05_bs_2_maxep_6_s2604_bart.csv',
 'log_bart_abs_2409_0144_lr_3e_05_bs_4_maxep_6.csv',
 'test_metrics_bart_abs_2409_0144_lr_3e_05_bs_2_maxep_6_s2604_bart.csv']

In [28]:
FINISHED_EXPERIMENTS = []
for exp in EXPERIMENT_PARAMS:
    exp_name = f"lr_{exp['lr']}_bs-{exp['batch_size']}_maxep_{exp['epochs']}".replace('-','_')
    files_found = sum([(exp_name in filename) for filename in os.listdir(results_dir)])
    if files_found == 3:
        # Experiment finished
        FINISHED_EXPERIMENTS.append(exp)
EXPERIMENT_PARAMS = [exp for exp in EXPERIMENT_PARAMS if exp not in FINISHED_EXPERIMENTS]
print("Experiments left:", len(EXPERIMENT_PARAMS))

Experiments left: 15


In [29]:
def run_post_training(split, test_details, test_df_temp: pd.DataFrame, tokenizer, experiment, run_name_model, epoch, results_dir):
    # First line added due to label error, see 
    predictions = np.where(test_details.predictions != -100, test_details.predictions, tokenizer.pad_token_id)
    preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    test_df_temp['response'] = preds
    exp_res = {**experiment, **(test_details.metrics)}
    test_metrics_df = pd.DataFrame([exp_res])
    test_df_temp = test_df_temp.convert_dtypes()
    test_metrics_df = test_metrics_df.convert_dtypes()
    wandb.log({run_name_model: test_details.metrics})
    preds_name = f"{split}_preds_{run_name_model.replace('-','_')}_s{epoch}_bart.csv"
    metrics_name =  f"{split}_metrics_{run_name_model.replace('-','_')}_s{epoch}_bart.csv"
    test_df_temp.to_csv(os.path.join(results_dir, preds_name), index=False, header=False, encoding='utf-8', quoting=csv.QUOTE_ALL)
    test_metrics_df.to_csv(os.path.join(results_dir, metrics_name), index=False, header=True, encoding='utf-8', quoting=csv.QUOTE_ALL)

In [30]:
class ExtraCallback(TrainerCallback):        
    def on_train_end(self, args, state, control, **kwargs):
        # Save and upload CSVs
        super().on_train_end(args, state, control, **kwargs)
        df = pd.DataFrame(state.log_history)
        df = df.convert_dtypes()
        df = df.groupby(['epoch'], as_index=False).sum()
        df.to_csv(os.path.join(results_dir, "log_" + args.run_name.replace('-','_') + ".csv"), header=True, index=False)

In [31]:
for count, exp in enumerate(EXPERIMENT_PARAMS):
    run_name_model = f"{run_name}-lr-{exp['lr']}-bs-{exp['batch_size']}-maxep-{exp['epochs']}"
    print("=== Starting experiment", count, f"on {get_current_time()}:", run_name_model, "training")
    wandb.run.name = run_name_model
    wandb.run.save()

    training_args = Seq2SeqTrainingArguments(
        output_dir=os.path.join(models_dir, run_name_model),
        eval_strategy="epoch",
        logging_strategy="epoch",
        save_only_model=True,
        learning_rate=exp['lr'],
        per_device_train_batch_size=exp['batch_size'],
        per_device_eval_batch_size=exp['batch_size'],
        weight_decay=0.1,
        save_strategy="epoch",
        save_total_limit=1,
        load_best_model_at_end=True,
        metric_for_best_model="eval_rouge/rougeL",
        greater_is_better=True,
        num_train_epochs=exp['epochs'],
        predict_with_generate=True,
        fp16=True,
        generation_max_length=80,
        push_to_hub=False,
        report_to="wandb",
        run_name=run_name_model,
    )
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_tweetsumm_abs["train"], # .select(range(0,50)),
        eval_dataset=tokenized_tweetsumm_abs["validation"], # .select(range(0,10)),
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics_abs,
    )
    trainer.add_callback(ExtraCallback)
    training_start = time.time()
    trainer.train()
    training_end = time.time()
    print(f"Finished experiment {count}: {run_name_model} - time it took for training:", str(datetime.timedelta(seconds=(training_end-training_start))))
    test_details = trainer.predict(tokenized_tweetsumm_abs['test'], metric_key_prefix='test')
    run_post_training('test', test_details, test_df, tokenizer, exp, run_name_model, trainer.state.best_model_checkpoint.split('-')[-1], results_dir)
    trainer.push_to_hub()
    shutil.rmtree(models_dir)
    os.makedirs(models_dir)

wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


=== Starting experiment 0 on 2409-1949: bart-abs-2409-1947-lr-3e-05-bs-2-maxep-10 training


/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss,Rouge/rouge1,Rouge/rouge2,Rouge/rougel,Rouge/rougelsum,Bertscore/bertscore-precision,Bertscore/bertscore-recall,Bertscore/bertscore-f1,Meteor,Gen Len
1,2.329900,2.199738,0.448400,0.212700,0.384200,0.385600,0.893900,0.890400,0.892000,0.410700,38.427273
2,1.617400,2.058076,0.461700,0.215800,0.391900,0.393000,0.897200,0.890700,0.893800,0.404900,36.054545
3,1.185400,2.188786,0.463100,0.215300,0.389300,0.391400,0.900000,0.891000,0.895300,0.404400,35.827273
4,0.856800,2.377647,0.451800,0.201000,0.382200,0.383900,0.897300,0.889500,0.893200,0.390700,34.763636
5,0.605600,2.646757,0.473100,0.220700,0.403100,0.403600,0.897400,0.894600,0.895900,0.417300,37.581818
6,0.423500,2.922561,0.481600,0.225800,0.405500,0.406400,0.898900,0.895500,0.897000,0.431700,36.890909
7,0.299500,3.193845,0.454100,0.198900,0.383900,0.384300,0.894100,0.891600,0.892700,0.407100,37.009091
8,0.207900,3.417814,0.468400,0.209400,0.392600,0.393100,0.894100,0.894200,0.894000,0.417700,39.454545
9,0.152700,3.534687,0.471600,0.215100,0.398900,0.400900,0.893700,0.894200,0.893800,0.427700,39.590909
10,0.120300,3.654172,0.473300,0.222800,0.409000,0.410300,0.895700,0.894500,0.894900,0.425700,38.072727


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

Finished experiment 14: bart-abs-2409-1947-lr-3e-06-bs-8-maxep-10 - time it took for training: 0:19:59.770202


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 62, 'min_length': 11, 'early_stopping': True, 'num_beams': 6, 'length_penalty': 0.5, 'no_repeat_ngram_size': 3, 'forced_eos_token_id': 2}


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.50k [00:00<?, ?B/s]

In [32]:
# Using wandb documentation: https://docs.wandb.ai/guides/artifacts
def log_csv_wandb(results_path, model_name):
    artifact = wandb.Artifact(name=model_name, type="predictions")
    for root, dirs, files in os.walk(results_path):
        for file in files:
            artifact.add_file(local_path=os.path.join(root, file), name=file)
    wandb.log_artifact(artifact)

In [33]:
log_csv_wandb(results_dir, run_name)

In [34]:
print("Finished all training and evaluation for", run_name)
wandb.finish()

Finished all training and evaluation for bart-abs-2409-1947


wandb:                                                                                
wandb: 
wandb: Run history:
wandb:        eval/bertscore/bertscore-f1 ▇██▇▇█▄▃▃▂▂▁▃▃▄▂▃▃▂▁▃▃▃▂▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
wandb: eval/bertscore/bertscore-precision █████████▆▄▁▃▅▅▃▄▂▂▅▄▃▄▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
wandb:    eval/bertscore/bertscore-recall ▇████████▃▂▂▃▂▂▃▂▃▃▃▂▃▁▃▄▄▃▃▃▃▄▄▄▄▄▄▄▃▄▃
wandb:                       eval/gen_len ▄▄▄▄▅▄▄▄▄▄▃▅▄▆▃██▇▂▂▆▃▁▆▅▄▆▆▆▆▆▆▄▄▄▄▄▄▆▄
wandb:                          eval/loss ▁▁▃▃▃▃▃▃▄▅▆▅▆▇▆▆▇▇▆▆▇▇▇█▇▇▇▇▇███████████
wandb:                        eval/meteor █▇▇▇▇▂▂▂▁▁▂▁▃▂▂▂▂▂▂▃▃▃▃▃▃▃▂▂▂▂▃▂▂▂▃▃▃▂▂▂
wandb:                  eval/rouge/rouge1 ▇█▇▇▇█▁▂▁▂▁▂▂▃▂▁▃▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▃▃▂▂▂▃▂▂
wandb:                  eval/rouge/rouge2 █▇█▇▇▇▇▇▇▂▁▂▁▂▁▂▁▁▁▂▂▂▁▃▁▂▂▂▂▂▃▃▃▂▃▃▂▃▃▃
wandb:                  eval/rouge/rougeL ▇▇▇▇██▂▂▃▂▁▂▂▁▁▂▂▁▁▃▂▂▃▁▃▂▂▂▂▂▃▃▃▂▃▃▂▂▁▃
wandb:               eval/rouge/rougeLsum ▇▇█▇█▇█▇███▄▂▃▁▃▃▁▃▁▂▃▃▃▃▂▂▂▂▃▃▃▃▃▃▃▂▂▂▂
wandb:                       eval/runtime █▅▅▅▃▃▃▄▃▃▄▅▃

In [35]:
print("Results uploaded")

Results uploaded
